# Advection constrain
<div class="alert alert-block alert-info">
ℹ️ This notebook describes the use of an advection constrain in the gridding.<br>
This constrains allows one to take into account a velocity field. 
</div>

In [ ]:
import Pkg
Pkg.activate("../..")
Pkg.instantiate()
using DIVAnd
using Makie, CairoMakie, GeoMakie
using Statistics
include("../config.jl")

## Function to interpolate

In [ ]:
fun(x, y) = sin.(6x) * cos.(6y)

# observations
x = rand(1);
y = rand(1);
f = fun.(x, y)

# final grid
xi, yi = ndgrid(range(0, stop = 100, length = 100), range(0, stop = 110, length = 110));

# reference field
fref = fun.(xi, yi);

### Simple plot

In [ ]:
fig = Figure(size = (500, 500))
ax = Axis(fig[1, 1])
heatmap!(ax, xi[:, 1], yi[1, :], fref)
fig

## Create mask

In [ ]:
mask = trues(size(xi));

# Add false along the edges
mask[1, :] .= false;
mask[end, :] .= false;
mask[:, 1] .= false;
mask[:, end] .= false;

# island
mask[30:80, 30:80] .= false;

### Simple plot
Note the `false` values along the borders.

In [ ]:
fig = Figure(size = (500, 500))
ax = Axis(fig[1, 1])
contourf!(ax, xi[:, 1], yi[1, :], mask, levels = [-0.5, 0.5])
fig

## Set metrics and analysis parameters

In [ ]:
# Metrics
pm = ones(size(xi)) / (xi[2, 1] - xi[1, 1]);
pn = ones(size(xi)) / (yi[1, 2] - yi[1, 1]);

# correlation length
len = 10;

# obs. error variance normalized by the background error variance
epsilon2 = 10000000.0;

## Advection along isobaths
`h` is the artificial topography, it has the same size as the computation grid 

In [ ]:
h = xi .* (100 .- xi) .+ 20;
@info(size(h))

fig = Figure(size = (550, 500))
ax = Axis(fig[1, 1])
hm = heatmap!(ax, xi[:, 1], yi[1, :], h)
Colorbar(fig[1, 2], hm)
fig

### Advection through velocity field
We define the 2 components of the velocity field field.     
`rfluxes` specifies the error variance on the constraint.    

In [ ]:
fluxes1 = sin.(yi[1, :] ./ 10.0) + 0.1 * rand(size(h)[2])
fluxes2 = sin.(xi[:, 1] ./ 10.0) + 0.1 * rand(size(h)[1])
rfluxes = 1;

In [ ]:
@info(size(fluxes1), size(fluxes2));

## Analysis
The first arguments of `DIVAndrun` are the same as a classical analysis.     
Then we add:
* `topographyforfluxes`: a tuple of 2 elements, one for each horizontal component, for the advection along isobath.
* `fluxes`: a tuple of 2 elements storing the pseudo-velocity field.
* `epsfluxes`: a scalar defining the error variance on the constraint.
### Run with one component

In [ ]:
@time fi, s = DIVAndrun(
    mask,
    (pm, pn),
    (xi, yi),
    (x, y),
    f,
    len,
    epsilon2;
    topographyforfluxes = (h, 0),
    fluxes = (fluxes1, 0),
    epsfluxes = rfluxes,
    alphabc = 1,
    alpha = [1, 0, 1],
);

#### Plot

In [ ]:
fig = Figure(size = (550, 500))
ax = Axis(fig[1, 1], title = "Interpolated field with advection")
hm = heatmap!(ax, xi[:, 1], yi[1, :], fi)
Colorbar(fig[1, 2], hm)
fig

### Run with the other component

In [ ]:
fluxesafter = zeros(size(h)[2])

for j = 1:size(h)[2]
    for i = 2:size(h)[1]-2
        if mask[i, j] && mask[i+1, j]
            fluxesafter[j] = fluxesafter[j] + h[i, j] * (fi[i+1, j] - fi[i, j])
        end
    end
end
@show var(fluxes1 + fluxesafter)
@show var(fluxes1)

@time fi, s = DIVAndrun(
    mask,
    (pm, pn),
    (xi, yi),
    (x, y),
    f,
    len,
    epsilon2;
    topographyforfluxes = (0, h),
    fluxes = (0, fluxes2),
    epsfluxes = rfluxes,
    alphabc = 1,
    alpha = [1, 0, 1],
);

#### Plot

In [ ]:
fig = Figure(size = (550, 500))
ax = Axis(fig[1, 1], title = "Interpolated field with advection")
hm = heatmap!(ax, xi[:, 1], yi[1, :], fi)
Colorbar(fig[1, 2], hm)
fig

### Finally using both components

In [ ]:
fluxesafter = zeros(size(h)[1])

for i = 1:size(h)[1]
    for j = 2:size(h)[2]-2
        if mask[i, j] && mask[i, j+1]
            fluxesafter[i] = fluxesafter[i] + h[i, j] * (fi[i, j+1] - fi[i, j])
        end
    end
end

@show var(fluxes2 + fluxesafter)
@show var(fluxes2)

# finally both directions
@time fi, s = DIVAndrun(
    mask,
    (pm, pn),
    (xi, yi),
    (x, y),
    f,
    len,
    epsilon2;
    topographyforfluxes = (h, h),
    fluxes = (fluxes1, fluxes2),
    epsfluxes = rfluxes,
    alphabc = 1,
    alpha = [1, 0, 1],
);

In [ ]:
fig = Figure(size = (550, 500))
ax = Axis(fig[1, 1], title = "Interpolated field with advection")
hm = heatmap!(ax, xi[:, 1], yi[1, :], fi)
Colorbar(fig[1, 2], hm)
fig

In [ ]:
fluxesafter = zeros(size(h)[2])

for j = 1:size(h)[2]
    for i = 2:size(h)[1]-2
        if mask[i, j] && mask[i+1, j]
            fluxesafter[j] = fluxesafter[j] + h[i, j] * (fi[i+1, j] - fi[i, j])
        end
    end
end
@show var(fluxes1 + fluxesafter)
@show var(fluxes1)

fluxesafter = zeros(size(h)[1])

for i = 1:size(h)[1]
    for j = 2:size(h)[2]-2
        if mask[i, j] && mask[i, j+1]
            fluxesafter[i] = fluxesafter[i] + h[i, j] * (fi[i, j+1] - fi[i, j])
        end
    end
end

@show var(fluxes2 + fluxesafter)
@show var(fluxes2);